# Scientific Computing: Diffusion Limited Aggregation and Reaction Diffusion

## Imports

In [1]:
import numpy as np
from IPython.display import HTML

import src.solutions as solutions
import src.visualizations as visualizations

In [2]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

## Diffusion Limited Aggregation

### Parameter Values

In [3]:
# packed parameters for easy transfer
N = 100
eta = 1
tol = 1e-4
maxiters = 10000
omega = 1.9
grid_indices = np.arange(N*N)
np.random.seed(22)
itertjes = 1500

etas = [0.4, 0.7, 1, 1.5, 2]
omegas = np.linspace(1.7, 2, 7)
omegas = np.delete(omegas, -1)

#### Initialization
creating an initial grid by placing object at the bottom of the 2D grid.   
The grid has a source at the top of the grid.   
within SOR, being part of the object is resembled with concentration of 0

In [4]:
object_grid = solutions.place_objects(N)
grid = solutions.initialize_grid(N, object_grid)
iters, grid = solutions.sequential_SOR(grid, tol, maxiters, omega, object_grid)
assert iters < maxiters, f"No convergence for SOR, omega: {omega} "

visualizing diffusion grid along with object (grid is turned 180 degrees)  
object is placed in 0 diffusion zone. 

In [ ]:
# normalized_grid = normalize_concentration(grid)
visualizations.plot_simulation_without_animation(grid, N, object_grid)
solutions.save_grid_to_file(grid)

#### Update initialized grid
run the DLA by solving the Laplace formula of the grid,  
calculating the cell connection probabilities with each cell concentration,  
and by selecting potential new cells joining the object (north, east, south, west) neighbors of the object. 
Update the grid by adding the selected cell to the object and rerunning these steps. 

Visualization of this process is provided here. 

In [ ]:
if omega != 1.9:
    omega = 1.9
    iters, grid = solutions.sequential_SOR(grid, tol, maxiters, omega, object_grid)     
    assert iters < maxiters, f"No convergence for SOR, omega: {omega} "
    
for eta in etas:
    iter_grid = np.copy(grid)
    object_grid_iter = np.copy(object_grid)
    Sr_pars = (tol, maxiters, omega)
    stencil_iter = solutions.generate_stencil(object_grid_iter)
    seedje = 22
    print(f"starting animation for DLA, eta: {eta} ")
    ani = visualizations.animate_1a(iter_grid, stencil_iter, object_grid_iter, grid_indices, eta, seedje, Sr_pars)


In [ ]:
eta =1

all_omega_iters = []
best_omegas = []
itertjes = 1500

for i in range(10): 
    best_omega = 0
    lowest_iter = np.inf
    omegas_iters = dict()

    print(f"starting experimentation for run {i} ")
    for omega in omegas:
        iter_grid = np.copy(grid)
        object_grid_iter = np.copy(object_grid)
        Sr_pars = (tol, maxiters, omega)
        stencil_iter = solutions.generate_stencil(object_grid_iter)
        seedje = 22
        
        total_sor_iters = 0
        for iter in range(itertjes):
            iter_grid, object_grid_iter, stencil_iter, sor_iters = solutions.perform_update_ADL(iter_grid, object_grid_iter, stencil_iter, grid_indices, eta, seedje, Sr_pars)
            total_sor_iters += sor_iters 
        if total_sor_iters < lowest_iter:
            lowest_iter = total_sor_iters
            best_omega = omega
        omegas_iters[omega] = total_sor_iters
    all_omega_iters.append(total_sor_iters)
    best_omegas.append(best_omega)